<img src="data/ima/herraa.jpg" alt="Imagen creada con inteligencia artificial y editada con Microsoft Paint" style="border-radius: 15px;">

*Imagen creada con inteligencia artificial y editada con Microsoft Paint*


## BIBLIOTECAS USADAS:

In [1]:
import numpy as np
import pandas as pd 
from scipy.stats import f_oneway 
import inspect

## DATASETS DE PRUEBA.

In [2]:
df1 = pd.read_csv('./data/datasets/lifesat_full.csv', sep=',')
df2 = pd.read_csv('./data/datasets/pima_indians.csv', sep=',') #Target 'class'
df3 = pd.read_csv('./data/datasets/temps.csv', sep=',')
df4 = pd.read_csv('./data/datasets/titanic.csv', sep=',') #Target 'alive'
df5 = pd.read_csv('./data/datasets/CarPrice_Assignment.csv', sep=',') # target 'Price'... o eso creo.  
df6 = pd.read_csv('./data/datasets/bank-full.csv', sep=';') #El target parce que es 'y'  
df7 = pd.read_csv('./data/datasets/dataset_viajes_jun23.csv', sep=',') #El target, 'aircompany' por ejemplo.  
df8 = pd.read_csv('./data/datasets/CarPrice_Assignment_manipu.csv', sep=',', encoding='latin1')



## Funcion: get_features_num_classification

In [3]:
def get_features_num_classification(df, target_col, p_value= 0.05): 
    columnas_num= [] 
    columnas_validas= []
    #limite = 1- p_value #No stoy seguro si hacerlo asi o directamente p_value

    df_name = [nombre for nombre, valor in globals().items() if valor is df][0] #Con esto convierto el nombre del dataframe en una variable

    if isinstance(df, pd.DataFrame):
        print(f"{df_name} es un DataFrame")
    else:
        print(f"El primer termino introducido no es un dataframe, repase la llamada a la función")
        return 

    if target_col not in df.columns:
        print(f"{target_col} no es una columna del dataframe df")
        return
    
    tipos_validos = ['category', 'int64', 'bool', 'object']
    tipo_columna = df[target_col].dtype
    if tipo_columna not in tipos_validos:
        print(f"La columna '{target_col}' no es categórica ni discreta, revisa la llamada a la funcion.")
        return 
    
    if pd.api.types.is_integer_dtype(df[target_col]) and df[target_col].nunique() >= 15: # Este "15" es el maximo a partir del cual avisa de la alta cardinalidad.
        print(f"¡OJO!.. La columna '{target_col}' tiene alta cardinalidad (> 15 categorías).")
    
    columnas_pre_num = df.select_dtypes(include=['number']).columns.tolist() 
    columnas_num= [] 
    
    for columna_numericas in columnas_pre_num:
        if columna_numericas != target_col: #Con esto nos aseguramos de no incluir el target.
            columnas_num.append(columna_numericas)
        elif columna_numericas == target_col:
            print("La columna objetivo no ha sido incluida en la lista.")
    
    if any(df[column].isnull().any() for column in columnas_num):
        print("ALGUNA DE SUS COLUMNAS NUMÉRICAS TIENE DATOS FALTANTES O ERRÓNEOS, LIMPIE SU DATAFRAME ANTES DE CONTINUAR")
        return
    
    for columna in columnas_num:
        grupos = []
        for categoria in df[target_col].unique():
            grupos.append(df[columna][df[target_col] == categoria])
    
        anova_result = f_oneway(*grupos)
        if anova_result.pvalue >= p_value: #posiblemnte sea oportuno usar 'limite'
            columnas_validas.append(columna)


    cantidad_elementos = len(columnas_validas)
    if cantidad_elementos == 0:
        print("NO HAY COLUMNAS QUE CUMPLAN LOS REQUISITOS")
        return
    if cantidad_elementos == 1:
        print("Solo una columna cumple los requisitos:")
        print(f"El valor de pvalue es {anova_result.pvalue}")
        return columnas_validas
    if cantidad_elementos > 1:
        print("Las columnas que cumplen requisitos son:")
        return columnas_validas


    


## COMPROBACIONES: 

In [4]:
no_df = ["esto", "es", "solo", 1, "ejemplo"] #Para comprobar si reconoce sin fallos lo que no son dataframe hemos hecho esto 
get_features_num_classification(no_df, 'target', p_value= 0.05)


El primer termino introducido no es un dataframe, repase la llamada a la función


In [5]:
get_features_num_classification(df2, 'mass', p_value= 0.05) #'mass' es una columna de float64

df2 es un DataFrame
La columna 'mass' no es categórica ni discreta, revisa la llamada a la funcion.


In [6]:
get_features_num_classification(df2, 'class', p_value= 0.05)

df2 es un DataFrame
La columna objetivo no ha sido incluida en la lista.
Solo una columna cumple los requisitos:
El valor de pvalue es 2.2099754606650332e-11


['pres']

In [7]:
get_features_num_classification(df5, 'price', p_value= 0.05)

df5 es un DataFrame
La columna 'price' no es categórica ni discreta, revisa la llamada a la funcion.


In [8]:
get_features_num_classification(df7, 'aircompany', p_value= 0.05)

df7 es un DataFrame
NO HAY COLUMNAS QUE CUMPLAN LOS REQUISITOS


In [9]:
get_features_num_classification(df6, 'y', p_value= 0.05)

df6 es un DataFrame
NO HAY COLUMNAS QUE CUMPLAN LOS REQUISITOS


In [10]:
get_features_num_classification(df4, 'alive', p_value= 0.05)

df4 es un DataFrame
Solo una columna cumple los requisitos:
El valor de pvalue es 6.120189341921873e-15


['sibsp']

In [11]:
get_features_num_classification(df8, 'fueltype', p_value= 0.05)

df8 es un DataFrame
ALGUNA DE SUS COLUMNAS NUMÉRICAS TIENE DATOS FALTANTES O ERRÓNEOS, LIMPIE SU DATAFRAME ANTES DE CONTINUAR
